<a href="https://colab.research.google.com/github/aagamaar/Algo_Trading_Python/blob/main/AlgoPy_1ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MOUNTED GOOGLE DRIVE

In [1]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')
print("Google Drive mounted successfully!")

Mounted at /content/drive
Google Drive mounted successfully!


CREATING PROJECT DIRECTORY STRUCTURE

In [2]:
# Cell 2: Create Project Directory Structure
import os

# Define your project root in Google Drive
project_root = '/content/drive/MyDrive/algo_trading_prototype'

# Create the main project directory
os.makedirs(project_root, exist_ok=True)
print(f"Project root created: {project_root}")

# Create subdirectories
subdirectories = [
    'config',
    'data',
    'strategy',
    'backtester',
    'analytics',
    'sheets',
    'utils',
    'logs' # Directory for log files
]

for subdir in subdirectories:
    path = os.path.join(project_root, subdir)
    os.makedirs(path, exist_ok=True)
    print(f"Created directory: {path}")

# Add __init__.py files to make directories into Python packages
# This is crucial for Python to recognize your folders as modules for importing.
for pkg_dir in ['data', 'strategy', 'backtester', 'analytics', 'sheets', 'utils']:
    with open(os.path.join(project_root, pkg_dir, '__init__.py'), 'w') as f:
        pass # Create an empty __init__.py file
    print(f"Created __init__.py in {pkg_dir}")

print("\nDirectory structure created successfully.")

Project root created: /content/drive/MyDrive/algo_trading_prototype
Created directory: /content/drive/MyDrive/algo_trading_prototype/config
Created directory: /content/drive/MyDrive/algo_trading_prototype/data
Created directory: /content/drive/MyDrive/algo_trading_prototype/strategy
Created directory: /content/drive/MyDrive/algo_trading_prototype/backtester
Created directory: /content/drive/MyDrive/algo_trading_prototype/analytics
Created directory: /content/drive/MyDrive/algo_trading_prototype/sheets
Created directory: /content/drive/MyDrive/algo_trading_prototype/utils
Created directory: /content/drive/MyDrive/algo_trading_prototype/logs
Created __init__.py in data
Created __init__.py in strategy
Created __init__.py in backtester
Created __init__.py in analytics
Created __init__.py in sheets
Created __init__.py in utils

Directory structure created successfully.
